# What to do with Missing Not At Random observations: using PyMC3's Bound variables to perform Bayesian imputation of censored data

In [ ]:
# set matplotlib backend to display plots inline
%matplotlib notebook

# imports
import numpy as np
import pandas as pd
import pymc3 as pm
import seaborn as sns
import scipy.stats

# resize plots to fit labels inside bounding box
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})

# MPI color scheme
sns.set_palette('Set2')

## Some helper functions for statistical modeling

In [ ]:
# standardize cf. Gelman recommendation (to get scale comparable to unstandardized binary variables)
def standardize(df):
    return (df - df.mean()) / (2 * df.std())

# specify and run model
def forestplot(model, transform=np.array, **kwargs):
    pm.forestplot(model.backend.trace,
                  varnames=model.fixed_terms.keys(),
                  transform=transform,
                  **kwargs)
    g = plt.gca()
    g.set(xlim=(None, None))
    no_effect = float(transform(0))
    g.axes.axvline(no_effect, color='red')
    g.axes.annotate('no effect', [no_effect, 0], rotation=90, va='top', ha='right', color='red')
    
# get posterior mode
def posterior_mode(trace):
    def _posterior_mode(samples, bins=500):
        samples = samples[np.isfinite(samples)]
        xmin = np.min(samples)
        xmax = np.max(samples)
        kernel = scipy.stats.gaussian_kde(samples)
        density = kernel.pdf(np.linspace(xmin, xmax, bins))
        step = (xmax - xmin) / bins
        return xmin + np.argmax(density) * step
    return np.apply_along_axis(_posterior_mode, 0, trace)

# add posterior mode to summary by default
def summary(trace, **kwargs):
    return pm.summary(trace,
                      extend=True,
                      stat_funcs=[lambda x: pd.Series(posterior_mode(x), name='mode')],
                      **kwargs)

# compare models
def compare(models, **kwargs):
    for model in models:
        model.backend.model.name = ' + '.join(model.terms.keys())
    models = {model.backend.model: model.backend.trace for model in models}
    comparison = pm.compare(models, **kwargs)
    pm.compareplot(comparison)
    return comparison

## Simulating some data
For this simple demonstration, we'll use two normally distributed predictors and a normally distributed error term to construct our observed variable.  

In [ ]:
randomnormal = np.random.normal(0, 5, (1000, 3))
df_sim = pd.DataFrame(randomnormal, columns=['x1', 'x2', 'e'])
df_sim['y'] = 30 + df_sim['x1'] + (df_sim['x2'] * 2) + df_sim['e']
g = sns.distplot(df_sim['y'])
g.set(title='y observed');

## Censoring the data
 Just imagine we're in a place where the average temperature is a nice warm 30 degrees, with some normally distributed variability. We have a thermometer that tops out at 40 degrees, so we'll cut our observed variable off at 40. That means occasionally our thermometer will indicate 40, but we know that it's quite likely to be warmer. We just don't know how much warmer, exactly.

In [ ]:
cutoff_right = 40
df_sim_obs = df_sim.loc[df_sim['y'] < cutoff_right]
g = sns.distplot(df_sim_obs['y'])
g.set(title='y observed without censored observations');

In [ ]:
df_sim_cens = df_sim.loc[df_sim['y'] > cutoff_right]
g = sns.distplot(df_sim_cens['y'])
g.set(title='y observed censored observations only');

## Modeling y observed without the censored observations
We're setting deliberately weak priors; it doesn't matter much here.

In [ ]:
with pm.Model() as sim_model:
    
    intercept = pm.Normal('Intercept', mu=0, sd=1.0)
    x1 = pm.Normal('beta_x1', mu=0, sd=1.0)
    x2 = pm.Normal('beta_x2', mu=0, sd=1.0)
    
    mu = intercept + (x1 * df_sim_obs['x1']) + (x2 * df_sim_obs['x2'])
    sigma = pm.HalfNormal('y_sd', sd=1.0)

    observed = pm.Normal('y', mu=mu, sd=sigma, observed=df_sim_obs['y'])

    print('Named variables being sampled:')
    [print(f'\t{key}') for key in sorted(list(sim_model.named_vars.keys()))]
    sim_model_trace = pm.sample(5000, tune=1000, init='advi', n_init=50000, chains=2, cores=2)

In [ ]:
g = pm.traceplot(sim_model_trace);

In [ ]:
display(summary(sim_model_trace).head())

Clearly, the intercept and the coefficients for both x1 and x2 are being underestimated.  
The values we were expecting are 30, 1.0, and 2.0, respectively, and the posterior modes are all about 10% too low.
We're also underestimating the variance of our observed values: the standard deviation of y is actually 5.

## Modeling y observed while imputing censored observations
We'll sample the censored observations from a separate distribution that is itself a parameter of the model.  

In [ ]:
with pm.Model() as sim_model_imputed:
    
    intercept = pm.Normal('Intercept', mu=0, sd=1.0)
    x1 = pm.Normal('x1', mu=0, sd=1.0)
    x2 = pm.Normal('x2', mu=0, sd=1.0)
    
    mu_cens = intercept + (x1 * df_sim_cens['x1']) + (x2 * df_sim_cens['x2'])
    mu = intercept + (x1 * df_sim_obs['x1']) + (x2 * df_sim_obs['x2'])
    sigma = pm.HalfNormal('y_sd', sd=1.0)

    right_censored = pm.Bound(pm.Normal, lower=cutoff_right)('right_censored', mu=mu_cens, sd=sigma, shape=len(df_sim_cens))
    observed = pm.Normal('y', mu=mu, sd=sigma, observed=df_sim_obs['y'])
    
    print('Named variables being sampled:')
    [print(f'\t{key}') for key in sorted(list(sim_model_imputed.named_vars.keys()))]
    sim_model_imputed_trace = pm.sample(5000, tune=1000, init='advi', n_init=50000, chains=2, cores=2)

In [ ]:
 g = pm.traceplot(sim_model_imputed_trace)

In [ ]:
display(summary(sim_model_imputed_trace).head(n=10))

Overall, the coefficient estimates are much better. We're still underestimating the average temperature (Intercept) a little, but all the posterior modes are close to the parameters we used to simulate the data.

## Model checks
The imputation has produced good posteriors, but as we will now demonstrate, the unconventional sampling breaks some of the methods we would normally use to check our model fit.

In [ ]:
# posterior predictive check
ppc = pm.sample_posterior_predictive(sim_model_imputed_trace, samples=10000, model=sim_model_imputed)
g = sns.distplot([n.mean() for n in ppc['y']])

That looks terrible, but that's because the posterior predictive distribution only includes the observations that weren't censored. The observed mean that is plotted here is inflated by the uncensored observations in the original dataset.  
Let's include the imputed values. These can be pulled from the sampling trace.

In [ ]:
g = sns.distplot([n.mean() for n in sim_model_imputed_trace['right_censored']])

Putting the posterior predictive and the imputed y values together gives us our proper y distribution.  
This becomes clear when we overlay the mean of the uncensored observed y values.

In [ ]:
g = sns.distplot([np.mean(np.hstack([a, b])) for a, b in zip(ppc['y'], sim_model_imputed_trace['right_censored'])])

# plot uncensored data mean line
line_x = df_sim['y'].mean()
g.axes.axvline(line_x, color='red')
g.axes.annotate(' observed mean', [line_x, 0], rotation=90, va='bottom', ha='right', color='red')
g.set(title='posterior predictive distribution');

In [ ]:
g = sns.scatterplot(x=ppc['y'].mean(axis=0), y=df_sim_obs['y'])
g.set(title='scatterplot', xlabel='predicted', ylabel='observed');

The scatterplot looks flattened at the top right, as we would expect because the values over 40 are censored.  
Even better than this scatterplot would be a quantile-quantile plot, but neither seaborn nor PyMC3 have that functionality built in, so we'll just write our own.

In [ ]:
# quantile-quantile plot function
def qqplot(predicted, observed, n=101):
    n = np.linspace(0, 100, n)
    x = np.percentile(predicted, n)
    y = np.percentile(observed, n)
    g = sns.scatterplot(x, y)
    lower = np.min([x, y])
    upper = np.max([x, y])
    g.axes.plot((lower, upper), (lower, upper), color='red')
    g.set(title='quantile-quantile plot', xlabel='predicted', ylabel='observed')
    return g
    
g = qqplot(ppc['y'].mean(axis=0), df_sim_obs['y'])

If we put the posterior predicted y values and the imputed y values together again, we can extend the q-q plot into the censored range.

In [ ]:
g = qqplot(np.hstack([ppc['y'].mean(axis=0), sim_model_imputed_trace['right_censored'].mean(axis=0)]),
           np.hstack([df_sim_obs['y'], df_sim_cens['y']]))

This still looks a little wonky around the censoring point (40) and just above it, but it seems to be doing okay overall.